### 1. Import packages

In [1]:
import deepdish as dd
memory = dd.io.load('fix_memory.h5')
print(len(memory))

15400


In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: rant3. Use `wandb login --relogin` to force relogin


True

In [3]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym

### 2. Helper functions

In [4]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    return state

In [5]:
def dict2array_partial(state):
    new_state = []
    num_observable_states = 0
    for key  in state.keys():
        if key != 'sw':
            if key == 'cumsumfert':
                new_state.append(state[key])
            if key == 'dap':
                new_state.append(state[key])
            if key == 'dtt':
                new_state.append(state[key])
            if key == 'istage':
                new_state.append(state[key])
            if key == 'pltpop':
                new_state.append(state[key])
            if key == 'rain':
                new_state.append(state[key])
            if key == 'srad':
                new_state.append(state[key])
            if key == 'tmax':
                new_state.append(state[key])
            if key == 'tmin':
                new_state.append(state[key])
            if key == 'vstage':
                new_state.append(state[key])
            if key == 'xlai':
                new_state.append(state[key])
        else:
            new_state += list(state['sw'])
    state = np.asarray(new_state)
    return state

### 3. Initialize the environment

In [6]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = 20
nb_actions = 25
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8 dimension
# how to defind nb_action? why is 200?

Observation: {'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}
27 9

Ram information received from DASSAT will has 20 dimensions.
There are 25 possible actions at each step.
Discrete? False


### 4. Define the network

In [7]:
class Full_QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=128*2,fc2_units=128*2,fc3_units=128*2):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(Full_QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, fc3_units)
        self.fc4 = nn.Linear(fc3_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        y = F.relu(self.fc2(x))
        z = F.relu(self.fc3(y))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return self.fc4(z)

In [8]:
trained_model = Full_QNetwork(35, 25)
trained_model.load_state_dict(torch.load('/home/rant3/focal/Model/FL BOTH Full/model2097.pth'))

<All keys matched successfully>

In [9]:
class Full_Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, trained_model):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size
        # Q-Network
        self.qnetwork = trained_model
        #self.qnetwork_local = QNetwork(state_size, action_size).to(device)

        # Replay memory

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork.eval()
        with torch.no_grad():
            action_values = self.qnetwork(state)
        self.qnetwork.train()

#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.nump
    def get(self,state):
        self.qnetwork.eval()
        with torch.no_grad():
            output= self.qnetwork(state)
        return output

In [10]:
class Partial_QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=128,fc2_units=128,fc3_units=128):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(Partial_QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, fc3_units)
        self.fc4 = nn.Linear(fc3_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        y = F.relu(self.fc2(x))
        z = F.relu(self.fc3(y))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return torch.round(24*torch.sigmoid(self.fc4(z)))

In [19]:
class Partial_QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=128):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(Partial_QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return torch.round(24*torch.sigmoid(self.fc2(x)))

In [20]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, LR):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size
        # Q-Network
        self.qnetwork = Partial_QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork.parameters(), lr=LR)
    
    
    def learn(self,sp,input1,episode):
        partial_sp=dict2array_partial(sp)
        partial_sp = torch.from_numpy(partial_sp).float().unsqueeze(0).to(device)
        input2=self.qnetwork(partial_sp)
        #print('input1 is', input1.item())
        #print('input2 is', input2)
        if round(input2.item())!= 0:
            print('input1 is', input1.item())
            print('input2 is', input2.item())
            print(episode)
            print('')
        loss = F.mse_loss(input1, input2)
        wandb.log({"loss": loss.item()})
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        # after this, the parameter of local network will change based on gradient descent
        for param in self.qnetwork.parameters():
            param.grad.data.clamp_(-1, 1)
        # stabilize traning to keep grad between (-1,1)
        self.optimizer.step()
    def save(self,name):
        torch.save(self.qnetwork.state_dict(),'/home/rant3/focal/model'+name+'.pth')

In [21]:
partial_agent = Agent(state_size=ram_dimensions, action_size=1, LR = 1e-5)
full_agent= Full_Agent(35,25,trained_model)

In [22]:
def super_partial(episodes,memory, exp=1):
    wandb.init(
      # Set the project where this run will be logged
      project="Super-Partial-single-output", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{exp}",)
    loss_window=deque(maxlen=1000)
    loss_list=[]
    for i in range(1, episodes+1):
        sp = random.choice(memory)
        #print(sp)
        full_sp=dict2array(sp)
        #full_sp = torch.from_numpy(full_sp).float().unsqueeze(0).to(device)
        input1=full_agent.act(full_sp,0)
        input1=torch.tensor([[float(input1)]])
        #input1=torch.tensor([[float(0)]])
        partial_agent.learn(sp,input1,i)
        #print('loss is',loss)
        #loss_list.append(loss.item())
        #loss_window.append(loss.item())
        #if i % 1000 == 0:
            #print('Average loss is', np.mean(loss_window))
        partial_agent.save('1')
            #partial_agent.save(str(i))

In [23]:
super_partial(10000,memory)

input1 is 0.0
input2 is 2.0
1

input1 is 0.0
input2 is 1.0
2

input1 is 0.0
input2 is 11.0
4

input1 is 0.0
input2 is 8.0
5

input1 is 0.0
input2 is 11.0
6

input1 is 0.0
input2 is 6.0
7

input1 is 0.0
input2 is 17.0
9

input1 is 0.0
input2 is 3.0
10

input1 is 0.0
input2 is 15.0
11

input1 is 0.0
input2 is 8.0
12

input1 is 0.0
input2 is 1.0
13

input1 is 0.0
input2 is 7.0
15

input1 is 0.0
input2 is 12.0
16

input1 is 0.0
input2 is 9.0
17

input1 is 0.0
input2 is 1.0
19

input1 is 0.0
input2 is 2.0
20

input1 is 0.0
input2 is 4.0
21

input1 is 0.0
input2 is 6.0
22

input1 is 0.0
input2 is 6.0
23

input1 is 0.0
input2 is 8.0
24

input1 is 0.0
input2 is 4.0
25

input1 is 0.0
input2 is 1.0
26

input1 is 0.0
input2 is 11.0
28

input1 is 0.0
input2 is 19.0
29

input1 is 0.0
input2 is 1.0
30

input1 is 0.0
input2 is 7.0
31

input1 is 0.0
input2 is 19.0
32

input1 is 0.0
input2 is 4.0
33

input1 is 0.0
input2 is 1.0
34

input1 is 0.0
input2 is 1.0
35

input1 is 0.0
input2 is 14.0
36

input1

input1 is 0.0
input2 is 10.0
297

input1 is 0.0
input2 is 1.0
298

input1 is 0.0
input2 is 4.0
299

input1 is 0.0
input2 is 4.0
301

input1 is 0.0
input2 is 1.0
303

input1 is 0.0
input2 is 3.0
304

input1 is 0.0
input2 is 5.0
305

input1 is 0.0
input2 is 14.0
306

input1 is 0.0
input2 is 14.0
308

input1 is 0.0
input2 is 12.0
309

input1 is 0.0
input2 is 1.0
310

input1 is 0.0
input2 is 8.0
314

input1 is 0.0
input2 is 12.0
315

input1 is 0.0
input2 is 4.0
316

input1 is 0.0
input2 is 2.0
317

input1 is 0.0
input2 is 14.0
319

input1 is 0.0
input2 is 9.0
321

input1 is 0.0
input2 is 5.0
322

input1 is 0.0
input2 is 1.0
323

input1 is 0.0
input2 is 4.0
324

input1 is 0.0
input2 is 2.0
325

input1 is 0.0
input2 is 2.0
326

input1 is 0.0
input2 is 2.0
327

input1 is 0.0
input2 is 12.0
328

input1 is 0.0
input2 is 11.0
329

input1 is 6.0
input2 is 2.0
331

input1 is 0.0
input2 is 14.0
332

input1 is 0.0
input2 is 1.0
333

input1 is 0.0
input2 is 12.0
334

input1 is 0.0
input2 is 1.0
335



input1 is 0.0
input2 is 8.0
599

input1 is 0.0
input2 is 12.0
600

input1 is 0.0
input2 is 1.0
601

input1 is 0.0
input2 is 17.0
603

input1 is 0.0
input2 is 2.0
604

input1 is 0.0
input2 is 1.0
605

input1 is 0.0
input2 is 14.0
606

input1 is 0.0
input2 is 8.0
607

input1 is 0.0
input2 is 1.0
608

input1 is 0.0
input2 is 1.0
609

input1 is 0.0
input2 is 11.0
610

input1 is 0.0
input2 is 5.0
613

input1 is 0.0
input2 is 1.0
614

input1 is 0.0
input2 is 1.0
615

input1 is 0.0
input2 is 1.0
616

input1 is 0.0
input2 is 11.0
617

input1 is 0.0
input2 is 2.0
618

input1 is 0.0
input2 is 14.0
619

input1 is 0.0
input2 is 7.0
620

input1 is 0.0
input2 is 14.0
621

input1 is 0.0
input2 is 6.0
622

input1 is 0.0
input2 is 2.0
624

input1 is 0.0
input2 is 2.0
625

input1 is 0.0
input2 is 4.0
626

input1 is 0.0
input2 is 2.0
627

input1 is 0.0
input2 is 11.0
630

input1 is 0.0
input2 is 8.0
631

input1 is 0.0
input2 is 3.0
633

input1 is 0.0
input2 is 5.0
634

input1 is 0.0
input2 is 1.0
635

in

input1 is 0.0
input2 is 2.0
899

input1 is 0.0
input2 is 11.0
900

input1 is 0.0
input2 is 13.0
901

input1 is 0.0
input2 is 5.0
902

input1 is 0.0
input2 is 7.0
903

input1 is 0.0
input2 is 10.0
904

input1 is 0.0
input2 is 14.0
905

input1 is 0.0
input2 is 10.0
906

input1 is 0.0
input2 is 20.0
907

input1 is 0.0
input2 is 4.0
908

input1 is 0.0
input2 is 8.0
909

input1 is 0.0
input2 is 4.0
910

input1 is 0.0
input2 is 11.0
911

input1 is 0.0
input2 is 2.0
912

input1 is 0.0
input2 is 2.0
913

input1 is 6.0
input2 is 3.0
914

input1 is 0.0
input2 is 2.0
915

input1 is 0.0
input2 is 12.0
916

input1 is 0.0
input2 is 20.0
917

input1 is 0.0
input2 is 2.0
919

input1 is 0.0
input2 is 19.0
921

input1 is 0.0
input2 is 2.0
922

input1 is 0.0
input2 is 12.0
923

input1 is 0.0
input2 is 1.0
924

input1 is 0.0
input2 is 5.0
925

input1 is 0.0
input2 is 2.0
926

input1 is 0.0
input2 is 13.0
927

input1 is 0.0
input2 is 11.0
928

input1 is 0.0
input2 is 1.0
929

input1 is 0.0
input2 is 4.0
93

input1 is 0.0
input2 is 6.0
1215

input1 is 0.0
input2 is 14.0
1216

input1 is 0.0
input2 is 7.0
1217

input1 is 0.0
input2 is 2.0
1220

input1 is 0.0
input2 is 12.0
1221

input1 is 0.0
input2 is 1.0
1222

input1 is 0.0
input2 is 1.0
1223

input1 is 0.0
input2 is 14.0
1224

input1 is 6.0
input2 is 2.0
1225

input1 is 0.0
input2 is 5.0
1226

input1 is 0.0
input2 is 1.0
1228

input1 is 0.0
input2 is 6.0
1229

input1 is 0.0
input2 is 14.0
1230

input1 is 0.0
input2 is 12.0
1231

input1 is 0.0
input2 is 6.0
1232

input1 is 0.0
input2 is 13.0
1233

input1 is 0.0
input2 is 1.0
1234

input1 is 0.0
input2 is 6.0
1235

input1 is 0.0
input2 is 8.0
1236

input1 is 0.0
input2 is 7.0
1237

input1 is 0.0
input2 is 1.0
1238

input1 is 0.0
input2 is 1.0
1239

input1 is 0.0
input2 is 2.0
1240

input1 is 0.0
input2 is 10.0
1241

input1 is 0.0
input2 is 4.0
1242

input1 is 0.0
input2 is 1.0
1243

input1 is 0.0
input2 is 14.0
1244

input1 is 0.0
input2 is 19.0
1245

input1 is 0.0
input2 is 3.0
1246

input

...

input1 is 0.0
input2 is 23.0
8569

input1 is 0.0
input2 is 1.0
8571

input1 is 0.0
input2 is 1.0
8572

input1 is 0.0
input2 is 5.0
8573

input1 is 0.0
input2 is 14.0
8574

input1 is 0.0
input2 is 14.0
8575

input1 is 0.0
input2 is 6.0
8576

input1 is 0.0
input2 is 13.0
8578

input1 is 0.0
input2 is 5.0
8579

input1 is 0.0
input2 is 7.0
8582

input1 is 0.0
input2 is 8.0
8583

input1 is 0.0
input2 is 6.0
8584

input1 is 0.0
input2 is 3.0
8585

input1 is 0.0
input2 is 1.0
8586

input1 is 0.0
input2 is 19.0
8587

input1 is 0.0
input2 is 13.0
8588

input1 is 6.0
input2 is 2.0
8589

input1 is 0.0
input2 is 1.0
8590

input1 is 0.0
input2 is 11.0
8591

input1 is 0.0
input2 is 1.0
8592

input1 is 0.0
input2 is 1.0
8593

input1 is 0.0
input2 is 4.0
8595

input1 is 6.0
input2 is 2.0
8596

input1 is 0.0
input2 is 4.0
8597

input1 is 0.0
input2 is 2.0
8598

input1 is 0.0
input2 is 2.0
8599

input1 is 0.0
input2 is 2.0
8600

input1 is 0.0
input2 is 5.0
8601

input1 is 0.0
input2 is 8.0
8602

input1 

input2 is 2.0
8856

input1 is 0.0
input2 is 2.0
8859

input1 is 0.0
input2 is 1.0
8860

input1 is 0.0
input2 is 6.0
8861

input1 is 0.0
input2 is 2.0
8863

input1 is 0.0
input2 is 1.0
8864

input1 is 0.0
input2 is 1.0
8865

input1 is 0.0
input2 is 5.0
8866

input1 is 0.0
input2 is 11.0
8868

input1 is 0.0
input2 is 13.0
8870

input1 is 0.0
input2 is 2.0
8871

input1 is 0.0
input2 is 5.0
8872

input1 is 0.0
input2 is 14.0
8873

input1 is 0.0
input2 is 15.0
8874

input1 is 0.0
input2 is 9.0
8875

input1 is 0.0
input2 is 3.0
8878

input1 is 0.0
input2 is 1.0
8879

input1 is 0.0
input2 is 5.0
8880

input1 is 0.0
input2 is 12.0
8881

input1 is 0.0
input2 is 2.0
8882

input1 is 0.0
input2 is 5.0
8886

input1 is 6.0
input2 is 1.0
8887

input1 is 0.0
input2 is 7.0
8888

input1 is 0.0
input2 is 1.0
8890

input1 is 0.0
input2 is 12.0
8891

input1 is 0.0
input2 is 10.0
8893

input1 is 0.0
input2 is 3.0
8894

input1 is 0.0
input2 is 5.0
8896

input1 is 0.0
input2 is 2.0
8897

input1 is 0.0
input2 

input1 is 0.0
input2 is 11.0
9159

input1 is 0.0
input2 is 14.0
9160

input1 is 0.0
input2 is 13.0
9161

input1 is 0.0
input2 is 14.0
9162

input1 is 0.0
input2 is 1.0
9163

input1 is 0.0
input2 is 13.0
9164

input1 is 0.0
input2 is 2.0
9165

input1 is 0.0
input2 is 2.0
9166

input1 is 0.0
input2 is 13.0
9167

input1 is 0.0
input2 is 12.0
9168

input1 is 0.0
input2 is 23.0
9169

input1 is 0.0
input2 is 1.0
9170

input1 is 0.0
input2 is 14.0
9171

input1 is 0.0
input2 is 4.0
9172

input1 is 0.0
input2 is 14.0
9173

input1 is 0.0
input2 is 11.0
9174

input1 is 0.0
input2 is 12.0
9175

input1 is 0.0
input2 is 7.0
9176

input1 is 0.0
input2 is 12.0
9177

input1 is 0.0
input2 is 2.0
9178

input1 is 0.0
input2 is 1.0
9179

input1 is 0.0
input2 is 4.0
9180

input1 is 0.0
input2 is 1.0
9182

input1 is 0.0
input2 is 1.0
9183

input1 is 0.0
input2 is 5.0
9184

input1 is 0.0
input2 is 6.0
9185

input1 is 0.0
input2 is 1.0
9188

input1 is 0.0
input2 is 2.0
9189

input1 is 0.0
input2 is 1.0
9190

i

input1 is 0.0
input2 is 3.0
9465

input1 is 0.0
input2 is 2.0
9466

input1 is 6.0
input2 is 3.0
9467

input1 is 0.0
input2 is 11.0
9468

input1 is 0.0
input2 is 6.0
9469

input1 is 0.0
input2 is 7.0
9471

input1 is 0.0
input2 is 12.0
9472

input1 is 0.0
input2 is 2.0
9473

input1 is 0.0
input2 is 14.0
9474

input1 is 0.0
input2 is 10.0
9475

input1 is 6.0
input2 is 2.0
9476

input1 is 0.0
input2 is 23.0
9477

input1 is 0.0
input2 is 11.0
9479

input1 is 0.0
input2 is 2.0
9480

input1 is 0.0
input2 is 4.0
9481

input1 is 0.0
input2 is 6.0
9482

input1 is 0.0
input2 is 11.0
9483

input1 is 0.0
input2 is 10.0
9484

input1 is 0.0
input2 is 4.0
9485

input1 is 0.0
input2 is 7.0
9486

input1 is 0.0
input2 is 7.0
9487

input1 is 0.0
input2 is 4.0
9488

input1 is 0.0
input2 is 13.0
9489

input1 is 0.0
input2 is 8.0
9490

input1 is 0.0
input2 is 5.0
9491

input1 is 0.0
input2 is 2.0
9492

input1 is 0.0
input2 is 8.0
9493

input1 is 0.0
input2 is 5.0
9494

input1 is 0.0
input2 is 2.0
9495

input

input1 is 0.0
input2 is 9.0
9743

input1 is 0.0
input2 is 1.0
9744

input1 is 0.0
input2 is 4.0
9745

input1 is 0.0
input2 is 2.0
9746

input1 is 0.0
input2 is 2.0
9748

input1 is 0.0
input2 is 1.0
9749

input1 is 0.0
input2 is 14.0
9751

input1 is 0.0
input2 is 19.0
9752

input1 is 0.0
input2 is 2.0
9753

input1 is 0.0
input2 is 5.0
9754

input1 is 0.0
input2 is 10.0
9755

input1 is 0.0
input2 is 11.0
9756

input1 is 0.0
input2 is 14.0
9757

input1 is 0.0
input2 is 4.0
9759

input1 is 0.0
input2 is 5.0
9760

input1 is 0.0
input2 is 9.0
9762

input1 is 6.0
input2 is 18.0
9763

input1 is 0.0
input2 is 9.0
9764

input1 is 0.0
input2 is 1.0
9765

input1 is 0.0
input2 is 5.0
9766

input1 is 0.0
input2 is 1.0
9768

input1 is 0.0
input2 is 1.0
9769

input1 is 0.0
input2 is 1.0
9770

input1 is 0.0
input2 is 5.0
9771

input1 is 0.0
input2 is 12.0
9772

input1 is 0.0
input2 is 10.0
9773

input1 is 0.0
input2 is 6.0
9774

input1 is 0.0
input2 is 23.0
9775

input1 is 0.0
input2 is 6.0
9776

input